Lecture 8 will cover polymorphism, inheritance and abstract methods in OOP.

Reference
 * [1] Chapter 10
 * [2] Section 9.5

# A bit of repetition from last lecture

### Clearifying the `self` argument

In [33]:
class MyClass:
    def __init__(self, a, b, c):
        self.d = a + b + c
    
    def do_thing(self, e, f):
        return self.d + e + f

In [34]:
x = MyClass(1, 2, 3)
# Calling a method on object x:
print( x.do_thing(5, 6) )
# is equivalent to
print( MyClass.do_thing(x, 5, 6) )

17
17


Other OOP languages do not have "self" as an argument. This is more a side-effect of how Python implements OOP. You aren't meant to call them like this under any normal circumstance.

### You almost always want the __init__ method

You probably want to do:
```python
h = Hand()
...
h.add_card(c)
...
```
So, you don't have all the cards from the start, and you don't have any input to the constructor.
You still want the `__init__` method!

In [3]:
class Hand:
    def __init__(self):
        self.cards = [] # We almost always want to intialize variables.

    def add_card(card):
        self.cards.append(card)

Note that an empty list is *not* some universal "empty" when declaring variables. An optional name might have a default as an empty string. If no sensible defualt value can be used, you should use `None`.

# An appetizer!

Simple inheritance in Python:

In [35]:
class Shape:
    def compute_area(self):
        # This is one common way to make an (optional) abstract method
        raise NotImplementedError("Missing compute_area implementation")

class Square(Shape): # The Square is a Shape
    def __init__(self, x0, h):
        self.x0 = x0
        self.h = h

    def compute_area(self): # Overloads the method from Shape
        return self.h**2
    
class Circle(Shape): # The Circle is a shape
    def __init__(self, x0, r):
        self.x0 = x0
        self.r = r

    def compute_area(self): # Overload the method from Shape
        from math import pi
        return pi * self.r**2

In [36]:
s = Square((0.,0.), 3.)
c = Circle((5.,3.), 1.)
print( s.compute_area() )
print( c.compute_area() )

9.0
3.141592653589793


# What is polymorphism?

In programming languages and type theory, polymorphism (from Greek πολύς, polys, "many, much" and μορφή, morphē, "form, shape") is the provision of a single interface to entities of different types. A polymorphic type is a type whose operations can also be applied to values of some other type, or types.

*Thanks Wikipedia!* 


## Parametric polymorphism

Also called generic (Java or C#) or template (C++ or D) programming. It means we can write general functions for many possible types. This is automatically the case for all code in Python due to the dynamic typing system (at a high runtime performance cost).

In [37]:
def max(a, b):
   return a if a > b else b

# Works for anything that supports the less-than operator:
print( max(3.4, 4.5) )
print( max('Hello', 'World') )

4.5
World


and we can compare this to a simple function in C
```c
float max(float a, float b) {
    return a > b ? a : b;
}
```
In C, we have to declare types (and we can't do parametric polymorphism without reverting to meta-programming)

In C++, we could do things like
```cpp
template< typename T >
T max(T a, T b) {
    return a > b ? a : b;
}
```

## Subtyping (or inclusion polymorphism)

In OOP, this is usually just called *polymorphism*, and is achieved by *inheritance*.

It might look like this:

In [38]:
class Animal:
    pass

class Dog(Animal):
    pass

class Bird(Animal):
    pass

class Hawk(Bird):
    pass

x = Hawk()
print("A hawk is a type of animal -", isinstance(x, Animal) )
print("A hawk is a type of bird   -", isinstance(x, Bird) )
print("A hawk is a type of dog    -", isinstance(x, Dog) )

A hawk is a type of animal - True
A hawk is a type of bird   - True
A hawk is a type of dog    - False


Python doesn't actually _require_ this type of polymorphism very much thanks to it's automatic parametric polymorphism, however, it is nice to have as it helps to express intent and it makes things easier, and, in some places, you can implement specialized behavior of a function based on the type information.

In statically typed langauges, this is the primary type of polymorphism used. As a very simple example:

```cpp
// This code is slightly simplified to not mention to many C++ details,
// but the point of polymorphism is the same
vector< Shape > shapes = {Triangle(...), Rectangle(...), ... };

float area = 0;
for ( Shape &s : shapes ) { // Loop over all the shapes
    area += s.computeArea() // All shapes are required to implement this method
}
printf("total area is %f\n", area);
```

### A note on the jargon

```python
class B(A):
    ...
```
* B is a **subclass** of A, and A is a **superclass** of B
* B is **derived** from the **base** class A

### Inherited functionality

Often the case is that you need to add another method to (or modify) an existing object, and keep all the other functionality intact.

In this example, we extend the standard `list` class with a new method `print_content` for convenience.

In [39]:
class MyList(list):
    def print_content(self):
        for i, x in enumerate(self):
            print("{}:".format(i), x)

In [40]:
x = MyList()
x.append("Hello")
x.append("World")
x.append("Some other string")
x.sort()
x.print_content()

0: Hello
1: Some other string
2: World


### Overloading methods

Derived classes (classes that inherit from some base-class) can mostly be seen as specialization of the general base class. Therefore, it is very common that methods need to be amended.

Very similar to the procedure of adding new methods (previous example), but we simply name the new method the same as the one we want to replace:

In [44]:
# class Fraction: <-- This is shorthand for Fraction(object):
class Fraction(object):
    def __init__(self, numerator: int, denominator: int):
        self.numerator = numerator
        self.denominator = denominator

    # Overloading __str__ from the object.__str__
    def __str__(self):
        return "{}/{}".format(self.numerator, self.denominator)

x = Fraction(3, 5)
print(x) # Print will call str(x) which calls x.__str__()

3/5


### The `super` function

In [58]:
class Base:
    def myMethod2(self):
        return "Fooxxx"

class Derived(Base):
    def myMethod2(self): return "asdf"
    def myMethod(self):
        # super() refers to the superclass (in this case, Base)
        return super().myMethod() + "Bar"

In [59]:
x = Derived()
print( x.myMethod() )

AttributeError: 'super' object has no attribute 'myMethod'

#### The super function with multiple inheritance

In [60]:
class Base:
    def my_method(self):
        return "Foo"
    
class Base2:
    def my_method(self):
        return "Bar"

class Derived(Base, Base2):
    def my_method(self):
        # super will refer to the first superclass that matches: Base
        return super().my_method() + "!"

In [61]:
x = Derived()
print( x.my_method() )

Foo!


In [62]:
class Derived(Base):
    def my_method(self):
        # If we want Base2.my_method, then we have to be specific
        return Base2.my_method(self) + "!"

### Enforced functionality (abstract methods)

Using the module `abc` we can get *abstract* methods:

In [111]:
import abc

class Pizza(metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def vegetarian(self):
        """ Returns true if vegetarian """

    @abc.abstractmethod
    def calories(self):
        """ Returns the number of calories in the pizza """

    def weight_watcher_compatible(self):
        return self.calories() <= 1000
        
class Margherita(Pizza):
    def calories(self):
        return 910
    
    def vegetarian(self):
        return True

class Vesuvio(Margherita):
    def calories(self):
        return 1100
    
    def vegetarian(self):
        return False


If we don't overload the abstract methods, then we get a TypeError:

In [118]:
class KebabPizza(Pizza):
    def vegetarian(self):
        return False

class JohannebergSpecial(KebabPizza):
    def calories(self):
        return 2100

class Instanbul(KebabPizza):
    def calories(self):
        return 2000

In [123]:
p = KebabPizza()  # This won't work

TypeError: Can't instantiate abstract class KebabPizza with abstract methods calories

In [122]:
p = JohannebergSpecial()  # This will work

Though in Python (due to the dynamic typing and Duck typing) the concept of base classes with abstract methods are not strictly necessary. The usual procedure is to just do:

In [105]:
class Pizza():
    def calories(self):
        """ Returns the number of calories in the pizza """
        # "raise" is a keyword for raising exceptions (throw) 
        # which aborts the program unless the caller cover the error handling.
        raise NotImplementedError("Derived class didn't overload this method")

Now, this will work fine:

In [110]:
class KebabPizza(Pizza):
    # Oops, forgot to overload "calories"!
    pass

p = KebabPizza()

As long as we don't try to call the missing method:

In [70]:
p.calories()

NotImplementedError: Derived class didn't overload this method

There is advantages and disadvantages to each way of doing this.

* Using the abc class, we get the error message immediately, which is much easier to debug.
* Using the NotImplementedError approach, we basically have an optional, abstract, method.

The `@abstractmethod` decorator is the closest equivalent to what you do in e.g. Java/C++

# Combining it all (examples)

Note: Examples below make use of operator overloading. E.g. `*` which becomes `__mul__`. Next lecture will cover, for now you can view them as methods. I use these in the examples because that's the most logical way to achieve it.

## Specialized matrices

Note: One would never implement matrices backed with a plain `list()` in Python. The purpose of this example is to show meaningful inheritance without complicating the implementation. This is dreadfully slow. 

In [87]:
import abc

class Matrix(metaclass=abc.ABCMeta):
    def __init__(self, nr, nc):
        self.nr = nr
        self.nc = nc
        if nr <= 0 or nc <= 0:
            raise ValueError("Bad nr or nc")

    @abc.abstractmethod
    def __mul__(self, val):
        """ does stuff """

    def bounds_check(self, item):
        if item[0] >= self.nr or item[1] >= self.nc:
            raise IndexError("Index out of range: ({},{})".format(*item))
        if item[0] < 0 or item[1] < 0:
            raise IndexError("Negative index not supported: ({},{})".format(*item))


class IdentityMatrix(Matrix):
    def __init__(self, size):
        super().__init__(size, size)

    def __str__(self):
        return "I({})".format(self.nr)

    def __getitem__(self, item):
        # This is called them you do:   a = x[i]
        self.bounds_check(item)
        return 1 if item[0] == item[1] else 0

    def __setitem__(self, item):
        # This is called them you do:   x[i] = a
        raise NotImplementedError("Identities are immutable")

    def __mul__(self, val):
        # This is called them you do:   x * y
        if type(val) == float or type(val) == int:
            return DiagonalMatrix([val]*self.nr)
        elif type(val) == list:
            return val.copy()
        else:
            raise TypeError("Unsupported type: {}".format(type(val)))

    def __rmul__(self, val):
        # This is called them you do:   y * x
        return self * val


class DiagonalMatrix(Matrix):
    def __init__(self, diag):
        super().__init__(len(diag), len(diag))
        self.diag = diag

    def __str__(self):
        s = "Diagonal matrix, size: {}x{}\nDiagonal: [".format(self.nr, self.nc)
        # Fancy formatting:
        v = [str(x) for x in self.diag]
        if len(self.diag) <= 10:
            s += ', '.join(v)
        else:
            s += ', '.join(v[:3]) + ' ... '+ ', '.join(v[-3:])
        return s + ']'

    def __getitem__(self, item):
        self.bounds_check(item)
        return self.diag[item[0]] if item[0] == item[1] else 0

    def __setitem__(self, item, val):
        self.bounds_check(item)
        if item[0] != item[1]:
            raise IndexError("Diagonal matrix only support setting diagonal values ({},{})".format(*item))
        self.diag[item[0]] = val

    def __mul__(self, val):
        if type(val) == float or type(val) == int:
            return DiagonalMatrix([d * val for d in self.diag])
        elif type(val) == list:
            if len(self.diag) != self.nr:
                raise ValueError('List not of correct size; {} should be {}'.format(len(val), self.nr))
            return [x*y for x, y in zip(self.diag, val)]
        else:
            raise TypeError("Unsupported type: {}".format(type(val)))

    def __rmul__(self, val):
        return self.__mul__(val)


class DenseMatrix(Matrix):
    def __init__(self, values):
        super().__init__(len(values), len(values[0]))
        self.values = values
        for i, row in enumerate(values):
            if len(row) != self.nc:
                raise MatrixSizeError(i, len(row), self.nc)

    def __str__(self):
        s = "Dense matrix, size: {}x{}\n".format(self.nr, self.nc)
        # Medium fancy formatting:
        for r in range(min(5, self.nr)):
            s += '['
            s += ', '.join([str(x) for x in self.values[r][:5]])
            if self.nc > 5:
                s += ', ...'
            s += "]\n"
        if self.nr > 5:
            s += '[...]\n'

        return s

    def __getitem__(self, item):
        self.bounds_check(item)
        return self.values[item[0]][item[1]]

    def __setitem__(self, item, val):
        self.bounds_check(item)
        self.values[item[0]][item[1]] = val

    def __mul__(self, val):
        if type(val) == float or type(val) == int:
            return DenseMatrix([[x * val for x in row] for row in self.values])
        elif type(val) == list:
            # Dot product reminder: y[i] = A_[i,j] * x_[j] !
            res = [[0.]*self.nc for x in range(self.nr)]
            for i in range(self.nr):
                for j in range(self.nc):
                    res[i][j] += self.values[i][j] * val[j]
            return res
        else:
            raise TypeError("Unsupported type: {}".format(type(val)))

    def __rmul__(self, val):
        # Copy paste is good enough for the exam! Identical to __mul__ but we switch the index
        if type(val) == float or type(val) == int:
            return DenseMatrix([[x * val for x in row] for row in self.values])
        elif type(val) == list:
            # Dot product reminder: y[j] = A_[i,j] * x_[i] !
            val = [[0.]*self.nc for x in range(self.nr)]
            for i in range(self.nr):
                for j in range(self.nc):
                    val[i][j] += self.values[j][i] * val[j]
            return val
        else:
            raise TypeError("Unsupported type: {}".format(type(val)))


class MatrixSizeError(Exception):
    def __init__(self, i, rowlen, ncol):
        self.i = i
        self.rowlen = rowlen
        self.ncol = ncol

    def __str__(self):
        return "Row number {} has {} columns, but should have {}.".format(self.i, self.rowlen, self.ncol)

In [88]:
# Test creation:
a = IdentityMatrix(3)
b1 = DiagonalMatrix([2.0, 3.5, 2.2, 2.2, 7.0])
b2 = DiagonalMatrix([2.0, 3.5, 2.2, 5.3, 4.3, 12.0, 2.2, 9.3, 1.2, 2.3, 2.2, 7.0])
c1 = DenseMatrix([[2.0, 0.0, 1.4],
                [0.5, 1.0, 1.0],
                [0.8, 3.0, 0.0]])

c2 = DenseMatrix([[2.0, 0.0, 1.4, 1, 2, 3, 4],
                [0.5, 1.0, 1.0, 1, 2, 3, 4],
                [0.8, 3.0, 0.0, 1, 2, 3, 4]])

c3 = DenseMatrix([[2.0, 0.0, 1.4],
                [0.5, 1.0, 1.0],
                [0.8, 3.0, 0.0],
                [0.8, 3.0, 0.0],
                [0.8, 3.0, 0.0],
                [0.8, 3.0, 0.0]])

In [89]:
# Test print:
print('a:', a)
print('b1:', b1)
print('b2:', b2)
print('c1:', c1)
print('c2:', c2)
print('c3:', c3)

a: I(3)
b1: Diagonal matrix, size: 5x5
Diagonal: [2.0, 3.5, 2.2, 2.2, 7.0]
b2: Diagonal matrix, size: 12x12
Diagonal: [2.0, 3.5, 2.2 ... 2.3, 2.2, 7.0]
c1: Dense matrix, size: 3x3
[2.0, 0.0, 1.4]
[0.5, 1.0, 1.0]
[0.8, 3.0, 0.0]

c2: Dense matrix, size: 3x7
[2.0, 0.0, 1.4, 1, 2, ...]
[0.5, 1.0, 1.0, 1, 2, ...]
[0.8, 3.0, 0.0, 1, 2, ...]

c3: Dense matrix, size: 6x3
[2.0, 0.0, 1.4]
[0.5, 1.0, 1.0]
[0.8, 3.0, 0.0]
[0.8, 3.0, 0.0]
[0.8, 3.0, 0.0]
[...]



In [90]:
# Test get-index:
print(a[2, 2], b1[2, 2], c1[2, 2])
print(a[1, 2], b1[1, 2], c1[1, 2])

1 2.2 0.0
0 0 1.0


In [91]:
# Test bounds checking
try:
    print(a[10,2])
except IndexError as e:
    print(e)

Index out of range: (10,2)


In [92]:
# Test a bit of set-index
b1[2, 2] = 3.14
c1[1, 2] = 3.14
print(b1)
print(c1)

Diagonal matrix, size: 5x5
Diagonal: [2.0, 3.5, 3.14, 2.2, 7.0]
Dense matrix, size: 3x3
[2.0, 0.0, 1.4]
[0.5, 1.0, 3.14]
[0.8, 3.0, 0.0]



In [42]:
# Test scaling.
print("a * 3 =", a * 1.5)
print("3 * b1 =", 1.5 * b1)
print("3 * c1 =", 1.5 * c1)

a * 3 = Diagonal matrix, size: 3x3
Diagonal: [1.5, 1.5, 1.5]
3 * b1 = Diagonal matrix, size: 5x5
Diagonal: [3.0, 5.25, 4.71, 3.3000000000000003, 10.5]
3 * c1 = Dense matrix, size: 3x3
[3.0, 0.0, 2.0999999999999996]
[0.75, 1.5, 4.71]
[1.2000000000000002, 4.5, 0.0]



In [45]:
# Test dot product
print("a * x =", a * [1, 2, 3])
print("x * b1 =", [1, 2, 3] * b1)
print("c1 * x =", c1 * [1, 2, 3])
try:
    print(c1 * "Hello")
except:
    print("Didn't work")

a * x = [1, 2, 3]
x * b1 = [2.0, 7.0, 9.42]
c1 * x = [[2.0, 0.0, 4.199999999999999], [0.5, 2.0, 9.42], [0.8, 6.0, 0.0]]
Didn't work


In [46]:
# Test the exception
try:
    m = DenseMatrix([[1,2,3],[4,5,6],[7,8],[9,10,11]])
except MatrixSizeError as e:
    print("You better fix row {}!".format(e.i))
    print(e)

You better fix row 2!
Row number 2 has 2 columns, but should have 3.


Classes for storing expressions analytically:

In [96]:
import abc, math

class Function(metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def derivative(self):
        pass

    def __neg__(self):
        return Negate(self)

    def __add__(self, other):
        return Add(self, other)

    def __mul__(self, other):
        return Product(self, other)


class Negate(Function):
    def __init__(self, fun):
        self.fun = fun

    def __call__(self, x):
        return -self.fun(x)

    def derivative(self):
        return Negate(self.fun.derivative())


class Add(Function):
    def __init__(self, f, g):
        self.f = f
        self.g = g

    def __call__(self, x):
        return self.f(x) + self.g(x)

    def derivative(self):
        return Add(self.f.derivative(), self.g.derivative())


class Product(Function):
    def __init__(self, f, g):
        self.f = f
        self.g = g

    def __call__(self, x):
        return self.f(x) * self.g(x)

    def derivative(self):
        return (self.f.derivative() * self.g) + (self.f.derivative() * self.g)


class Compose(Function):
    # Compose(f, g)(x) == f(g(x))
    def __init__(self, f, g):
        self.f = f
        self.g = g

    def __call__(self, x):
        return self.f(self.g(x))

    def derivative(self):
        return Compose(self.f.derivative(), self.g) * self.g.derivative()


class Sine(Function):
    def __call__(self, x):
        return math.sin(x)

    def derivative(self):
        return Cosine()


class Cosine(Function):
    def __call__(self, x):
        return math.cos(x)

    def derivative(self):
        return Sine()


class Constant(Function):
    def __init__(self, value):
        self.value = value

    def __call__(self, x):
        return self.value

    def derivative(self):
        return Constant(0)


class Pow(Function):
    def __init__(self, order):
        self.order = order

    def __call__(self, x):
        return x ** self.order

    def derivative(self):
        if self.order == 1:
            return Constant(1.)
        return Constant(self.order) * Pow(self.order-1)

In [97]:
f = -Sine() + Constant(5.7) + Constant(2) * Pow(3)  # f(x) = -sin(x) + 5.7 + 2*x**3
print("f(1.5)  =", f(1.5))
df = f.derivative()  # f'(x) = cos(x) + 6*x**2
print("f'(1.5) =", df(1.5))

g = Add(Add(Compose(Sine(), Cosine()), Negate(Cosine())), Constant(1)) # g(x) = sin(cos(x)) - cos(x) + 1
print("g(0.3)  =", g(0.3))
dg = g.derivative()  # g'(x) = cos(cos(x))*sin(x) + sin(x)
print("g'(0.3) =", dg(0.3))

h = ((-f) + g).derivative()
print("h(3.2)  =", h(3.2))

f(1.5)  = 11.452505013395946
f'(1.5) = -0.0707372016677029
g(0.3)  = 0.861171564168752
g'(0.3) = -0.12490633052658753
h(3.2)  = -0.9715440316334694


You will see another great example of inheritance in the third assignment: Overloading existing GUI widgets to create your own is an example where OOP shines. There are many other good examples, but many involve a lot of code, or domain specific expertise, to fully grasp; as such, these toy examples often seem a bit contrived.

In your coding career, do not introduce needless class hierarchies for the sake of it. Use them when they bring nicer structure, code safety, and simpler code.
Having said that; the exam will have 1 question that specifically cover OOP/inheritance in some form. There you should show be able to implement a few simple classes.